In [1]:
import torch
import torch.nn as nn
import numpy as np
from transformers import AutoTokenizer


c:\Users\fundacion\anaconda3\envs\transformers\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configuración inicial
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)


In [3]:
# Dataset de poesía simple
corpus = [
    "En el cielo las estrellas cantan",
    "La luna brilla con todo su esplendor",
    "Un río tranquilo acaricia la montaña",
    "El amor eterno nunca pierde su valor",
    "En el bosque los árboles susurran secretos",
    "Las flores bailan con el viento al pasar",
    "El sol se esconde dejando el mundo en calma",
    "Una melodía eterna resuena en el mar",
    "La brisa del amanecer despierta mi alma",
    "Los sueños se dibujan al borde del horizonte",
    "Un susurro de hojas acompaña mi caminar",
    "El río canta historias de un tiempo lejano",
    "Las estrellas observan el silencio de la noche",
    "El corazón late al ritmo de un poema eterno",
    "La lluvia besa la tierra en un dulce abrazo",
    "Un arco iris florece tras la tormenta",
    "El viento susurra secretos al oído del bosque",
    "Un pétalo cae, marcando el paso del tiempo",
    "El sol abraza el horizonte con su luz dorada",
    "Las nubes pintan sueños en el cielo infinito",
    "El mar guarda en su canto los misterios del mundo",
    "Un árbol solitario observa el paso de las estaciones",
    "La luna guía al viajero perdido en la noche",
    "El fuego danza al ritmo de un canto ancestral",
    "Las montañas guardan los secretos de los siglos",
    "El amor florece en el rincón más inesperado",
    "Una lágrima cae, reflejo de un alma perdida",
    "El tiempo esculpe la vida como un artista paciente",
    "El silencio de la madrugada guarda mil promesas",
    "Un poema se escribe en la brisa del atardecer",
    "La luz de la luna ilumina los senderos olvidados",
    "El eco de los sueños resuena en el valle",
    "Las estrellas caen como lágrimas del universo",
    "Un amanecer promete nuevos comienzos",
    "El agua del río limpia las heridas del alma",
    "La naturaleza canta una sinfonía de vida y muerte",
    "Un viajero solitario encuentra paz en la montaña",
    "La nostalgia se esconde en el murmullo del viento",
    "El cielo nocturno guarda los suspiros del pasado",
    "Las olas del mar susurran cuentos de sirenas",
    "El aroma del jazmín embriaga la noche",
    "Un atardecer rojo marca el fin de un día eterno",
    "La esperanza renace con cada amanecer",
    "El rocío decora las flores como lágrimas de la mañana",
    "Las sombras de los árboles bailan bajo la luna",
    "El amor eterno se graba en los anillos de un árbol",
    "Un sendero de flores guía al caminante perdido",
    "El canto de los pájaros despide al sol",
    "El crepúsculo tiñe el cielo con colores de fuego"
]



In [4]:
# Tokenizador de Hugging Face
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [5]:
# Configurar un token de relleno si no existe
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

vocab_size = tokenizer.vocab_size

In [6]:
# Preprocesamiento: convertir textos a índices de tokens
def encode_texts(corpus, max_length=20):
    encoded_texts = tokenizer(corpus, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
    return encoded_texts["input_ids"], encoded_texts["attention_mask"]

input_ids, attention_masks = encode_texts(corpus)


In [7]:
# Modelo Transformer simple para generación de texto
class PoetryTransformer(nn.Module):
    def __init__(self, vocab_size, emb_dim, n_heads, num_layers, ff_dim, max_seq_len):
        super(PoetryTransformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.positional_encoding = nn.Parameter(torch.zeros(1, max_seq_len, emb_dim))
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=emb_dim, nhead=n_heads, dim_feedforward=ff_dim, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.fc = nn.Linear(emb_dim, vocab_size)
    
    def forward(self, x, mask=None):
        embedded = self.embedding(x) + self.positional_encoding[:, :x.size(1), :]
        encoded = self.encoder(embedded, src_key_padding_mask=~mask)  # Invierte la máscara
        logits = self.fc(encoded)
        return logits


In [8]:
# Instanciar el modelo
emb_dim = 128
n_heads = 4
num_layers = 3
ff_dim = 256
max_seq_len = input_ids.size(1)

model = PoetryTransformer(vocab_size, emb_dim, n_heads, num_layers, ff_dim, max_seq_len).to(device)



In [9]:
# Configuración de entrenamiento
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [17]:
# Entrenamiento
epochs = 100
input_ids, attention_masks = input_ids.to(device), attention_masks.bool().to(device)

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(input_ids, mask=attention_masks)
    loss = criterion(outputs.view(-1, vocab_size), input_ids.view(-1))
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")


Epoch 1/100, Loss: 0.3194
Epoch 2/100, Loss: 0.2951
Epoch 3/100, Loss: 0.2739
Epoch 4/100, Loss: 0.2531
Epoch 5/100, Loss: 0.2365
Epoch 6/100, Loss: 0.2210
Epoch 7/100, Loss: 0.2054
Epoch 8/100, Loss: 0.1943
Epoch 9/100, Loss: 0.1805
Epoch 10/100, Loss: 0.1710
Epoch 11/100, Loss: 0.1586
Epoch 12/100, Loss: 0.1501
Epoch 13/100, Loss: 0.1423
Epoch 14/100, Loss: 0.1349
Epoch 15/100, Loss: 0.1287
Epoch 16/100, Loss: 0.1219
Epoch 17/100, Loss: 0.1160
Epoch 18/100, Loss: 0.1097
Epoch 19/100, Loss: 0.1053
Epoch 20/100, Loss: 0.1009
Epoch 21/100, Loss: 0.0971
Epoch 22/100, Loss: 0.0927
Epoch 23/100, Loss: 0.0887
Epoch 24/100, Loss: 0.0861
Epoch 25/100, Loss: 0.0822
Epoch 26/100, Loss: 0.0808
Epoch 27/100, Loss: 0.0775
Epoch 28/100, Loss: 0.0761
Epoch 29/100, Loss: 0.0740
Epoch 30/100, Loss: 0.0700
Epoch 31/100, Loss: 0.0685
Epoch 32/100, Loss: 0.0667
Epoch 33/100, Loss: 0.0647
Epoch 34/100, Loss: 0.0628
Epoch 35/100, Loss: 0.0610
Epoch 36/100, Loss: 0.0606
Epoch 37/100, Loss: 0.0587
Epoch 38/1

In [18]:
# Función de generación de texto
def generate_poetry(prompt, model, max_length=20):
    model.eval()
    prompt_ids = tokenizer(prompt, return_tensors="pt", padding="max_length", truncation=True, max_length=max_length)["input_ids"].to(device)
    
    generated = prompt_ids
    for _ in range(max_length - len(prompt_ids[0])):
        outputs = model(generated)
        next_token_logits = outputs[:, -1, :]
        next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
        generated = torch.cat((generated, next_token), dim=1)
        
        if next_token.item() == tokenizer.eos_token_id:
            break
    
    return tokenizer.decode(generated[0], skip_special_tokens=True)


In [19]:
# Generar poesía con un prompt
prompt = "En el mar"
poem = generate_poetry(prompt, model, 20)
print("\nGeneración de poesía:\n")
print(poem)



Generación de poesía:

en el mar
